In [3]:
import pandas as pd 
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn import preprocessing


In [5]:
train = pd.read_csv('/home/achintya/Desktop/CODES/ML Simplilearn/Demo Datasets/Lesson 4/bigmart_train.csv')
test = pd.read_csv('/home/achintya/Desktop/CODES/ML Simplilearn/Demo Datasets/Lesson 4/bigmart_test.csv')

In [7]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [9]:
train.shape

(8523, 12)

In [11]:
train.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [13]:
train['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [15]:
train['Outlet_Age']=2021 - train['Outlet_Establishment_Year']

In [17]:
train['Outlet_Size'].unique()

array(['Medium', nan, 'High', 'Small'], dtype=object)

In [19]:
train.describe

<bound method NDFrame.describe of      Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility  \
0              FDA15        9.300          Low Fat         0.016047   
1              DRC01        5.920          Regular         0.019278   
2              FDN15       17.500          Low Fat         0.016760   
3              FDX07       19.200          Regular         0.000000   
4              NCD19        8.930          Low Fat         0.000000   
...              ...          ...              ...              ...   
8518           FDF22        6.865          Low Fat         0.056783   
8519           FDS36        8.380          Regular         0.046982   
8520           NCJ29       10.600          Low Fat         0.035186   
8521           FDN46        7.210          Regular         0.145221   
8522           DRG01       14.800          Low Fat         0.044878   

                  Item_Type  Item_MRP Outlet_Identifier  \
0                     Dairy  249.8092            OUT04

In [21]:
train['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [23]:
train['Outlet_Size'].mode()

0    Medium
dtype: object

In [25]:
train['Outlet_Size']=train['Outlet_Size'].fillna(train['Outlet_Size'].mode()[0])

In [27]:
train['Item_Weight'] = train['Item_Weight'].fillna(train['Item_Weight'].mean())

In [29]:
train['Item_Visibility'].hist(bins = 20)

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [71]:
Q1 = train['Item_Visibility'].quantile(0.25)
Q3 = train['Item_Visibility'].quantile(0.75)
IQR = Q3 - Q1
filt_train = train.query('(@Q1 - 1.5 * @IQR) <= Item_Visibility <= (@Q3 + 1.5 * @IQR)')

In [72]:
filt_train.shape, train.shape

((8379, 13), (8523, 13))

In [73]:
train = filt_train


In [74]:
train['Item_Visibility'].unique()

array([0.0160473 , 0.01927822, 0.01676007, ..., 0.03518627, 0.14522065,
       0.04487828])

In [75]:
train['Item_Visibility_bins'] = pd.cut(train['Item_Visibility'], [0.000, 0.0650, 0.1300, 0.200], labels=['Low Viz', 'Viz', 'High Viz'])

In [76]:
train['Item_Visibility_bins'].head

<bound method NDFrame.head of 0        Low Viz
1        Low Viz
2        Low Viz
3            NaN
4            NaN
          ...   
8518     Low Viz
8519     Low Viz
8520     Low Viz
8521    High Viz
8522     Low Viz
Name: Item_Visibility_bins, Length: 8379, dtype: category
Categories (3, object): ['Low Viz' < 'Viz' < 'High Viz']>

In [77]:
train['Item_Visibility_bins'].isnull().sum()

526

In [78]:
train['Item_Visibility_bins'].value_counts()

Low Viz     4403
Viz         2557
High Viz     893
Name: Item_Visibility_bins, dtype: int64

In [79]:
train['Item_Visibility_bins'] = train['Item_Visibility_bins'].replace(np.nan, 'Low_Viz', regex=True)

In [80]:
train['Item_Fat_Content']=train['Item_Fat_Content'].replace(['low fat', 'LF', 'Low_Fat'], 'Low Fat')

In [81]:
train['Item_Fat_Content']=train['Item_Fat_Content'].replace(['reg' , 'Regular'], 'Regular_Fat')

In [82]:
le = preprocessing.LabelEncoder()

In [83]:
train['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular_Fat'], dtype=object)

In [84]:
train['Item_Fat_Content']=le.fit_transform(train['Item_Fat_Content'])

In [85]:
train['Item_Visibility_bins']=le.fit_transform(train['Item_Visibility_bins'])

In [86]:
train['Outlet_Size']=le.fit_transform(train['Outlet_Size'])

In [87]:
train['Outlet_Location_Type']=le.fit_transform(train['Outlet_Location_Type'])

In [88]:
dummy = pd.get_dummies(train['Outlet_Type'])

In [89]:
dummy.head()

,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
0,0,1,0,0
1,0,0,1,0
2,0,1,0,0
3,1,0,0,0
4,0,1,0,0


In [90]:
train = pd.concat([train, dummy], axis=1)

In [91]:
train.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales', 'Outlet_Age',
       'Item_Visibility_bins', 'Grocery Store', 'Supermarket Type1',
       'Supermarket Type2', 'Supermarket Type3'],
      dtype='object')

In [92]:
train = train.drop(['Item_Identifier', 'Item_Type', 'Outlet_Identifier', 'Outlet_Type', 'Outlet_Establishment_Year'], axis=1)

In [93]:
train.columns

Index(['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_MRP',
       'Outlet_Size', 'Outlet_Location_Type', 'Item_Outlet_Sales',
       'Outlet_Age', 'Item_Visibility_bins', 'Grocery Store',
       'Supermarket Type1', 'Supermarket Type2', 'Supermarket Type3'],
      dtype='object')

In [94]:
X = train.drop('Item_Outlet_Sales',axis=1)
y = train.Item_Outlet_Sales

In [95]:
test = pd.read_csv('/home/achintya/Desktop/CODES/ML Simplilearn/Demo Datasets/Lesson 4/bigmart_test.csv')

In [96]:
test['Outlet_Size'] = test['Outlet_Size'].fillna('Medium')

In [97]:
test.Item_Visibility

0       0.007565
1       0.038428
2       0.099575
3       0.015388
4       0.118599
          ...   
5676    0.013496
5677    0.142991
5678    0.073529
5679    0.000000
5680    0.104720
Name: Item_Visibility, Length: 5681, dtype: float64

In [98]:
test['Item_Visibility_bins'] = pd.cut(test['Item_Visibility'], [0.000, 0.065, 0.13, 0.2], labels=['Low Viz', 'Viz', 'High Viz'])

In [103]:
test['Item_Weight']=test['Item_Weight'].fillna(test['Item_Weight'].mean())

In [104]:
test['Item_Visibility_bins']=test['Item_Visibility_bins'].fillna('Low Viz')

In [107]:
test['Item_Visibility_bins'].head()

0    Low Viz
1    Low Viz
2        Viz
3    Low Viz
4        Viz
Name: Item_Visibility_bins, dtype: category
Categories (3, object): ['Low Viz' < 'Viz' < 'High Viz']

In [112]:
test['Item_Fat_Content']=le.fit_transform(test['Item_Fat_Content'])
test['Item_Visibility_bins']=le.fit_transform(test['Item_Visibility_bins'])
test['Outlet_Size']=le.fit_transform(test['Outlet_Size'])
test['Outlet_Location_Type']=le.fit_transform(test['Outlet_Location_Type'])
test['Outlet_Age'] = le.fit_transform(test['Outlet_Establishment_Year'])

In [115]:
dummy = pd.get_dummies(test['Outlet_Size'])
test = pd.concat([test,dummy], axis=1)

In [116]:
X_test = test.drop(['Item_Identifier', 'Item_Type', 'Outlet_Type', 'Outlet_Establishment_Year'], axis=1)

In [117]:
X.columns, X_test.columns

(Index(['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_MRP',
        'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Age',
        'Item_Visibility_bins', 'Grocery Store', 'Supermarket Type1',
        'Supermarket Type2', 'Supermarket Type3'],
       dtype='object'),
 Index([         'Item_Weight',     'Item_Fat_Content',      'Item_Visibility',
                    'Item_MRP',    'Outlet_Identifier',          'Outlet_Size',
        'Outlet_Location_Type', 'Item_Visibility_bins',           'Outlet_Age',
                             0,                      1,                      2],
       dtype='object'))

In [119]:
from sklearn import model_selection
xtrain, xtest, ytrain, ytest = model_selection.train_test_split(X, y, test_size=0.3, random_state=42)

In [121]:
lin = LinearRegression()

In [122]:
lin.fit(xtrain, ytrain)
print(lin.coef_)
print(lin.intercept_)

[    3.35428815    27.12327436  -105.837685      15.96668691
    16.78107616     8.03527797    -2.2746766     23.30477597
 -1754.40351049   219.65802578  -122.50086968  1657.24635438]
-202.93929347663834


In [123]:
predictions = lin.predict(xtest)


In [124]:
from math import sqrt


In [127]:
print(sqrt(mean_squared_error(ytest, predictions)))

1118.480514371628


In [128]:
from sklearn.linear_model import Ridge

In [134]:
ridgereg = Ridge(alpha=0.001, normalize=True)
ridgereg.fit(xtrain, ytrain)

Ridge(alpha=0.001, normalize=True)

In [136]:
print(sqrt(mean_squared_error(ytrain, ridgereg.predict(xtrain))))
print(sqrt(mean_squared_error(ytest, ridgereg.predict(xtest))))

1139.4537291881882
1118.4228456028761


In [139]:
print('R2 value/Coefficient of Determination: {} '.format(ridgereg.score(xtest, ytest)))

R2 value/Coefficient of Determination: 0.5486085198985001 


In [140]:
from sklearn.linear_model import Lasso
lassoreg = Lasso(alpha=0.001, normalize=True)

lassoreg.fit(xtrain, ytrain)


Lasso(alpha=0.001, normalize=True)

In [141]:
print(sqrt(mean_squared_error(ytrain, lassoreg.predict(xtrain))))
print(sqrt(mean_squared_error(ytest, lassoreg.predict(xtest))))

1139.4530688651178
1118.474132746509


In [142]:
print('R2 value/Coefficient of Determination: {} '.format(lassoreg.score(xtest, ytest)))

R2 value/Coefficient of Determination: 0.5485671203320125 


In [143]:
from sklearn.linear_model import ElasticNet
elas = ElasticNet(alpha = 0.001, normalize=True)
elas.fit(xtrain, ytrain)

print(sqrt(mean_squared_error(ytrain, elas.predict(xtrain))))
print(sqrt(mean_squared_error(ytest, elas.predict(xtest))))

1478.1842868516806
1429.0845842020078


In [144]:
print('R2 value/Coefficient of Determination: {} '.format(elas.score(xtest, ytest)))

R2 value/Coefficient of Determination: 0.2630175140598714 
